### Install Python Packages

Only execute during first run on new machine 

In [1]:
# Common python packages 
#pip install numpy
#pip install os

In [2]:
#pip install pymoo
#pip install pyfoam
#pip install ipynb
#pip install jdc

### Imports

In [3]:
# import RunOFv4.ipynb 
from ipynb.fs.full.RunOpenFOAMv4 import *

# Pymoo

### Problem
Runs OpenFOAM simulation for each individual in a generation.

OpenFOAM tutorial: incompresible/icoFoam/cavity

In [4]:
import numpy as np
import os
import io

from pymoo.util.misc import stack
from pymoo.model.problem import Problem

from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

#import RunOFv4

class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_constr=0,
                                    # mu_x  mu_y
                         xl=np.array([-0.3, 0]),
                         xu=np.array([-0.1, 0.15]))

    def _evaluate(self, x, out, *args, **kwargs):
        ###### Initialize Generation ######
        case = 'airfoil'
        gen = callback.gen
        
        solver = 'simpleFoam'
        mesher = 'blockMesh'
        
        # Maximum processors to be used 
        procLim = 4
        # Number of processors for each individual (EQUAL or SMALLER than procLim)
        nProc = 1
        
#         print('Gen: %i'%gen)
#         print('   mu_x       mu_y')
#         print(x)
        
        # create sim object for this generation and it's population
        sim = RunOFv4(case, x, gen, solver, mesher, procLim, nProc)
        
        obj = sim.runGen()
        
#         print('Objectives')
#         print('  Lift    Drag')
#         for i in range(len(obj[:][0])):
#             print('gen %i:'%i, end=' ')
#             for j in range(len(obj[0][:])):
#                 print('%.6f' %obj[i][j], end=' ')
#             print('\n')
        
        #out["F"] = np.column_stack([sigmaX, sigmaY])
        out['F'] = obj
            
        # objectives unconstrainted
        #g1 = 2*(x[:, 0]-0.1) * (x[:, 0]-0.9) / 0.18
        #g2 = - 20*(x[:, 0]-0.4) * (x[:, 0]-0.6) / 4.8
        #out["G"] = np.column_stack([g1, g2])
        
        
    # --------------------------------------------------
    # Pareto-front - not necessary but used for plotting
    # --------------------------------------------------
    def _calc_pareto_front(self, flatten=True, **kwargs):
        f1_a = np.linspace(0.1**2, 0.4**2, 100)
        f2_a = (np.sqrt(f1_a) - 1)**2

        f1_b = np.linspace(0.6**2, 0.9**2, 100)
        f2_b = (np.sqrt(f1_b) - 1)**2

        a, b = np.column_stack([f1_a, f2_a]), np.column_stack([f1_b, f2_b])
        return stack(a, b, flatten=flatten)

    # --------------------------------------------------
    # Pareto-set - not necessary but used for plotting
    # --------------------------------------------------
    def _calc_pareto_set(self, flatten=True, **kwargs):
        x1_a = np.linspace(0.1, 0.4, 50)
        x1_b = np.linspace(0.6, 0.9, 50)
        x2 = np.zeros(50)

        a, b = np.column_stack([x1_a, x2]), np.column_stack([x1_b, x2])
        return stack(a,b, flatten=flatten)

problem = MyProblem()

### Algorithm

In [5]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation

algorithm = NSGA2(
    pop_size=7,
    #n_offsprings=2,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

### Termination Criteria

from pymoo.factory import get_termination

termination = get_termination("n_gen", 2)

### Callback

In [6]:
from pymoo.model.callback import Callback

class MyCallback(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.gen=0
        self.data["best_obj1"] = []
        self.data['best_obj2'] = []
        self.data['var'] = []
        self.data['obj'] = []

    def notify(self, algorithm):
        self.data["best_obj1"].append(algorithm.pop.get("F")[:,0].min())
        self.data['best_obj2'].append(algorithm.pop.get('F')[:,1].min())
        self.data['var'].append(algorithm.pop.get('X'))
        self.data['obj'].append(algorithm.pop.get('F'))
        self.gen+=1
        
callback = MyCallback()

### Display

In [8]:
from pymoo.util.display import Display

class MyDisplay(Display):
    bestObj = []
    def _do(self, problem, evaluator, algorithm):
        super()._do(problem, evaluator, algorithm)
        #self.output.append("metric_a", np.mean(algorithm.pop.get("X")))
        #self.output.append("metric_b", np.mean(algorithm.pop.get("F")))
        self.output.append('Best Lift [N]', np.mean(algorithm.pop.get("F")[:,0].min()))
        self.output.append('Best Drag [N]', np.mean(algorithm.pop.get("F")[:,1].min()))
        
        #if

### Solve

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               ("n_gen", 5),
               callback=callback,
               seed=1,
               pf=problem.pareto_front(use_cache=False),
               save_history=True,
               display=MyDisplay(),
               verbose=True
              )

In [ ]:
print('Variables:')
print(callback.data['var'])
print('Objectives:')
print(callback.data['obj'])
print('Best Objective 1:')
for i in range(len(callback.data['best_obj1'])):
    print('%.6f' %callback.data['best_obj1'][i])
print('Best Objective 2:')
for i in range(len(callback.data['best_obj2'])):
    print('%.6f' %callback.data['best_obj2'][i])


In [ ]:
#print("Time Elapsed:")
#print(res.time)
print("Objectives:")
print(res.pop.get('F'))
#print(res.F)
print("Variables:")
print(res.X)

# Visualize Data

In [ ]:
%matplotlib inline 
# Figures are shown as static png images (optionally svg if configured)
# %matplotlib notebook #or %matplotlib nbagg - Interactive Figures inside the notebook
# %matplotlib widgets # Interactive Figures inside the notebook (requires jupyter-matplotlib to be installed)
# %matplotlib tk #or %matplotlib qt etc. - GUI windows show the figure externally to the notebook with the given interactive backend


In [ ]:
# the global factory method
from pymoo.factory import get_visualization
plot = get_visualization("scatter")

for gen in callback.data['obj']:
    #print(gen)
    plot.add(gen)

#plot.add(res.pop.get('F'), color="green", marker="x")
#print(res.pop.get('F'))
#plot.add(B, color="red", marker="*")
plot.show()

import numpy as np

A = np.random.random((20,2))
B = np.random.random((20,2))

from pymoo.factory import get_visualization
plot = get_visualization("scatter")
plot.add(A, color="green", marker="x")
plot.add(B, color="red", marker="*")
plot.show()

### Design + Objective Space

In [ ]:
from pymoo.visualization.scatter import Scatter

# get the pareto-set and pareto-front for plotting
ps = problem.pareto_set(use_cache=False, flatten=False)
pf = problem.pareto_front(use_cache=False, flatten=False)

# Design Space
plot = Scatter(title = "Design Space", axis_labels="x")
plot.add(res.X, s=30, facecolors='none', edgecolors='r')
plot.add(ps, plot_type="line", color="black", alpha=0.7)
plot.do()
plot.apply(lambda ax: ax.set_xlim(-0.5, 1.5))
plot.apply(lambda ax: ax.set_ylim(-2, 2))
plot.show()

# Objective Space
plot = Scatter(title = "Objective Space")
plot.add(res.F)
plot.add(pf, plot_type="line", color="black", alpha=0.7)
plot.show()

### Best Result vs. Generations

In [ ]:
import matplotlib.pyplot as plt
matplotlib.use('GTK3Agg')

val = [e.pop.get("F").min() for e in res.history]
plt.plot(np.arange(len(val)), val)
plt.show()